# GSS

In [ ]:
### gss算法原理
https://www.notion.so/GPU-GSS-9b62fcd7cdb24c59a106141e75c59f50
### gss代码解读
https://www.notion.so/GPU-GSS-9b1f500d10024238b2a2cfc3b352d193

# nns-car

## nns-car是实习项目中，公司使用的车载多音区pipeline。以下简述gss与nns-car的区别：
+ gss使用类似盲源分离的方法，估计出每个人的mask，然后使用基于语音、噪声互协方差矩阵的mvdr算法，实现语音分离
+ nns-car项目使用EaBnet（一个全nn的波束形成语音增强器，具有逐帧更新空间滤波器的特点），通过EaBnet生成的空间滤波器，
生成目标音区的语音信号，但由于逐帧输出空间滤波器工程不稳定的原因，会导致输出声音忽大忽小，质量堪忧，因此将EaBnet输出
的分离语音与输入noisey信号相除，得到mask。基于这个mask，引导bss正确输出各音区声道。

## 关于BSS算法
BSS算法中文名为盲信号分离，可以在没有语音与噪声的先验信息情况下，实现源分离，但依然需要源数量的先验信息。
假设多通道观测信号为x(t * c), 待估分离信号为y(t * m), 那么就有y = xM, M是混淆矩阵，维度是(c, m)。 

### BSS算法的PCA
PCA(主要成分分析)，首先对语音数据进行均值归零。然后计算语音信号的互协方差矩阵C。由于C是hermite的，因此可以对C进行特征矩阵分析，对C分解后，去前m个最大的特征值，以及对应的特征向量，组成m个特征子空间（PCA矩阵）。将原始信号投影到主分量基底组成的特征子空间上，就得到了一个源的信号。最后对各个源进行白化处理（使信号协方差矩阵单位对角化），去除其与其他信号分离之间的相关性。
注意这些源的信号仅仅只是线性不相关，在非线性条件下，依然可能相关，因此并不独立。  
PCA为ICA处理提供了很好的基础。

### BSS算法的ICA
ICA（独立成分分析）基于一个假设，即各个声源的概率分布是独立且非高斯的。一种优化方式是计算所有声源联合概率分布与各自概率分布乘积的kl散度，通过最大化kl散度（对比函数）来估计各个声源的概率分布。注意kl散度越大，两个概率分布的相似度越低。  
ICA 往往需要待分离信号是非高斯，根据**中心极限定理**，多个独立随机过程加和会逐渐接近高斯随机过程，如果信号源本身就是高斯的，那么它就很难满足独立分布的特点了。


## 关于MVDR算法
MVDR算法的核心是保持目标方向功率不变，将整体信号功率最大化，并优化得到空间滤波器W。  
最终目标方向的信号y = Wx。  
一般有两种方案：  
+ 将目标方向向量、噪声互协方差矩阵作为输入来获取空间滤波器
+ 将目标语音的互协方差矩阵、噪声的互协方差矩阵作为输入来获取空间协方差矩阵，这一点可以使用目标语音的mask实现（使用mask来估计语音、噪声的互协方差矩阵）。  
第一种形式的mvdr适用于强化特点方向的场景，第二种mvdr适用于不知道目标的方向，但知道目标语音先验信息从而实现语音分离的场景。

## eabnet：全nn波束成形器
这是项目中使用的语音增强网络，目的是根据多通道输入，输出一个类似mvdr的自适应的空间滤波器。这个空间滤波器与噪声在频域相乘并在通道维度相加即可得到最终的人声。

### EM
这是eabnet的作用是提取人声与噪声的特征，相当于mvdr算法所需要的人声与噪声的互协方差矩阵。主要由三部分组成：
+ $U^2$ Net编码层
+ TCN时序卷积网络
+ $U^2$ Net解码层
$U^2$Net的每一格编码、解码模块都不是简单的卷积神经网络，而是一个完整的RSU（残差U型神经网络），随着网络逐渐接近TCN，RSU的编解码结构层数逐渐减少，每个RSU的开头处，都有一个GLU。GLU灵感来自于LSTM，为线性层额外加入了一个门控，即独立增加一个可训练的权重矩阵V，线性计算后经过sigmoid函数来获取门控值，使更有价值的信息通过，低价值的信息被阻挡。TCN使用因果卷积、膨胀卷积，在时间轴上实现一维卷积，取代了传统RNN,在避免梯度消失、梯度爆炸、延时等RNN问题的情况下，提取了语音的序列信息。

### BF
这是eabnet通过EM输出的语音、噪声嵌入来实习波束成形的模块，类似于mvdr算法本身。分别有两种实现，第一种使用CNN，第二种是LSTM，最终逐帧生成自适应空间滤波器。空间滤波器与原始多通道信号卷积并在通道轴相加即可得到最终的增强语音。
